In [1]:
# Implied Volatility Finder


In [2]:
from libs import my_functions
import libs.universe as tickers
import pandas as pd
import pprint


In [3]:
start = '2023-10-01'
end = '2023-11-03'
tickers_list = tickers.Universe


In [4]:
result = my_functions.Alpaca_Markets.get_stock_data(tickers_list, '23Hour', start, end)
result


,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2023-10-01 13:00:00+00:00,12.81,13.010,12.80,152,12.94,24518,12.871946,AAL
2023-10-02 12:00:00+00:00,12.75,13.100,12.68,94715,12.84,37733713,12.811867,AAL
2023-10-03 11:00:00+00:00,12.30,12.800,12.25,112924,12.76,40449662,12.386147,AAL
2023-10-04 10:00:00+00:00,12.80,12.820,12.24,119606,12.31,49434802,12.611539,AAL
2023-10-05 09:00:00+00:00,12.82,13.010,12.72,94777,12.80,36246607,12.859079,AAL
...,...,...,...,...,...,...,...,...
2023-10-30 07:00:00+00:00,157.00,158.413,153.52,45958,156.90,2731232,156.699600,ZTS
2023-10-31 06:00:00+00:00,157.24,158.530,156.09,43975,157.51,3203166,156.992386,ZTS
2023-11-01 05:00:00+00:00,151.44,157.000,151.03,79857,156.00,7204270,152.634638,ZTS


In [5]:
iv_result = my_functions.calculate_volatility(result, tickers_list, '2023-11-01', end)
iv_resultsorted_iv = iv_result.sort_values(ascending = False).head(20)
iv_resultsorted_iv


Series([], dtype: float64)

In [6]:
'''
polygon_response = my_functions.Polygon_Markets.get_stock_data('AAPL', 1, 'day', '2023-11-01', '2023-11-03', 50000)
polygon_response
'''

"\npolygon_response = my_functions.Polygon_Markets.get_stock_data('AAPL', 1, 'day', '2023-11-01', '2023-11-03', 50000)\npolygon_response\n"

In [7]:
'''
polygon_options = my_functions.Polygon_Markets.get_options_data('F', '2023-11-01', '2023-11-03', 20, 150)
'''

"\npolygon_options = my_functions.Polygon_Markets.get_options_data('F', '2023-11-01', '2023-11-03', 20, 150)\n"

In [8]:
'''
opt = pprint.pprint(polygon_options)
'''



'\nopt = pprint.pprint(polygon_options)\n'

In [9]:
'''
import requests
import os
from dotenv import load_dotenv
from pprint import pprint
import json

load_dotenv()

polygon_key = os.getenv("POLYGON_API_KEY")

url = 'https://api.polygon.io/v3/snapshot/options/'
url += 'EVRI'
url+= f'?apiKey={polygon_key}'

res = requests.get(url)
res_formatted = res.json()
pprint(res_formatted)
'''

'\nimport requests\nimport os\nfrom dotenv import load_dotenv\nfrom pprint import pprint\nimport json\n\nload_dotenv()\n\npolygon_key = os.getenv("POLYGON_API_KEY")\n\nurl = \'https://api.polygon.io/v3/snapshot/options/\'\nurl += \'EVRI\'\nurl+= f\'?apiKey={polygon_key}\'\n\nres = requests.get(url)\nres_formatted = res.json()\npprint(res_formatted)\n'

In [10]:
'''
exp_date = res_formatted['results'][0]['details']['expiration_date']
strike = res_formatted['results'][0]['details']['strike_price']
type = res_formatted['results'][0]['details']['contract_type']
print(f'Expiration Date: {exp_date}\nStrike: {strike}\nContract Type: {type}')
'''

"\nexp_date = res_formatted['results'][0]['details']['expiration_date']\nstrike = res_formatted['results'][0]['details']['strike_price']\ntype = res_formatted['results'][0]['details']['contract_type']\nprint(f'Expiration Date: {exp_date}\nStrike: {strike}\nContract Type: {type}')\n"

In [14]:
import requests
import os
from dotenv import load_dotenv
from pprint import pprint
import json
from libs import universe
import pandas as pd
import logging
import time

load_dotenv()

symbol_list = universe.NYSE_COMM_SRVS
contract_type = 'PUT'
count = 4
to_date = '2024-03-15'

consumer_key = os.getenv('TDAMERITRADE_CONSUMER_KEY')
 
# Number of symbols to check per minute (TDA has a limit of calls per minute)
max_sym_limit = 100
# Number of seconds to pause and wait once max_sym_limit has been reached
seconds_to_pause = 65

data = {}
my_data = []

# Create logging object
logger = logging.getLogger(__name__)
logging.basicConfig(format='%(asctime)s %(message)s', filename='./exceptions.log', level=logging.INFO)

for i in range(len(symbol_list)):
    # Check if first symbol in list and ignore time delay
    if i != 0:
        # Check if maximum symbol limit is reached and delay GET Request
        if i % max_sym_limit == 0:
            time.sleep(seconds_to_pause)
    
    url = 'https://api.tdameritrade.com/v1/marketdata/chains?'
    url += f'apikey={consumer_key}'
    url += f'&symbol={symbol_list[i]}'
    url += f'&contractType={contract_type}'
    url += f'&strikeCount={count}'
    url += f'&toDate={to_date}'

    try:
        td_resp = requests.get(url)
    except Exception as e:
         logger.info(f'\nIssue is with response from GET request. {e}\nSymbol is:\n{symbol_list[i]}\n\n')

    symbol_data = td_resp.content.decode('ASCII')
    res_again = json.loads(symbol_data)
    try:
        if res_again["status"] == 'SUCCESS':
            data["symbol"] = res_again['symbol']
            data["underlying price"] = float(res_again['underlyingPrice'])
            data["expiration date"] = list(res_again['putExpDateMap'].keys())[0]
            data["strike price"] = float(list(res_again['putExpDateMap'][list(res_again['putExpDateMap'].keys())[0]].keys())[0])
            data["volatility"] = float(res_again['putExpDateMap'][list(res_again['putExpDateMap'].keys())[0]][list(res_again['putExpDateMap'][list(res_again['putExpDateMap'].keys())[0]].keys())[0]][0]['volatility'])
            data["bid"] = float(res_again['putExpDateMap'][list(res_again['putExpDateMap'].keys())[0]][list(res_again['putExpDateMap'][list(res_again['putExpDateMap'].keys())[0]].keys())[0]][0]['bid'])
            data["ask"] = float(res_again['putExpDateMap'][list(res_again['putExpDateMap'].keys())[0]][list(res_again['putExpDateMap'][list(res_again['putExpDateMap'].keys())[0]].keys())[0]][0]['ask'])
            data["delta"] = float(res_again['putExpDateMap'][list(res_again['putExpDateMap'].keys())[0]][list(res_again['putExpDateMap'][list(res_again['putExpDateMap'].keys())[0]].keys())[0]][0]['delta'])
            my_data.append(data.copy())

    except Exception as e:
            # Log exceptions and continue to next symbol
            logger.info(f'\nIssue that was thrown is related to data "status" check or data assignment to dictionary:\n{e}\nSymbol is:\n{symbol_list[i]}\n\n')
            continue



In [15]:
data_df = pd.DataFrame(my_data).set_index('symbol')
data_df.sort_values(by=['strike price'], ascending=False, inplace=True)
data_df.head(10)



,underlying price,expiration date,strike price,volatility,bid,ask,delta
symbol,,,,,,,
NFLX,437.500,2023-11-10:2,430.0,24.556,1.17,1.23,-0.224
CHTR,412.925,2023-11-10:2,410.0,23.667,1.70,1.95,-0.328
META,319.195,2023-11-10:2,315.0,25.617,0.96,1.02,-0.238
SPOT,169.735,2023-11-10:2,165.0,33.104,0.30,0.40,-0.141
MSGS,85.250,2023-11-17:9,160.0,27.693,0.20,0.40,-0.084
TMUS,145.820,2023-11-10:2,145.0,17.509,0.25,0.30,-0.217
TTWO,145.700,2023-11-10:2,142.0,72.041,2.72,2.88,-0.421
NXST,148.475,2023-11-17:9,140.0,36.258,0.20,1.80,-0.216
GOOG,133.000,2023-11-10:2,132.0,18.292,0.30,0.37,-0.263


In [16]:
volitility_min = 50.0
volitility_max = 75.0
price_max = 20.0
price_min = 5.0
strike_min = 1.0

data_filtered_greater_vol = data_df[data_df['volatility'] > volitility_min]
data_filtered_less_vol = data_filtered_greater_vol[data_filtered_greater_vol['volatility'] < volitility_max]
data_filtered_vol_less_price = data_filtered_less_vol[data_filtered_less_vol['underlying price'] < price_max]
data_filtered_vol_greater_price = data_filtered_vol_less_price[data_filtered_vol_less_price['underlying price'] > price_min]
data_filtered_vol_price_greater_strike = data_filtered_vol_greater_price[data_filtered_vol_greater_price['strike price'] > strike_min]
data_filtered = data_filtered_vol_price_greater_strike[data_filtered_vol_price_greater_strike['bid'] != 0.0]
data_filtered



,underlying price,expiration date,strike price,volatility,bid,ask,delta
symbol,,,,,,,
BILI,14.540,2023-11-10:2,14.0,55.945,0.07,0.09,-0.201
PARA,12.135,2023-11-10:2,11.0,55.522,0.01,0.02,-0.061
SNAP,11.205,2023-11-10:2,10.5,60.422,0.02,0.03,-0.091


In [ ]:
# Getting ready for API change from TDAmeritrade to Schwab

